In [ ]:
"""
Create a travel planner which checks the flight prices from city A to city B
within next 30 days and suggests cheapest flight for both to and fro journey
It can be customised if user wants only direct flights or indirects are fine too
"""

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [ ]:
import os
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [ ]:
# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
# Agent 1 - Flight Scraper

flight_scraper_agent = Agent(
    role = "Scrap flight details",
    goal = "Scrap latest flight prices and all other flight details "
            " like flight number, departure/arrival timings,  duration, no of stops from the internet. "
            " Make sure you check all popular Indian travel websites like makemytrip, easemytrip, adanione ",
    backstory = " I constantly travel for work and need to book cheapest flight satisfying all my constraints ",
    tools = [search_tool, scrape_tool],
    verbose = True,
    allow_delegation = False,
)


# Agent 2 - Builds itinerary ensuring that I stay for x days. It also prefers early morning flight while going 
# and evening flight while returning

itinerary_builder_agent = Agent(
    role = "Itinerary builder",
    goal = " Build a itinerary and suggest which flights I should take in my journey which satisfies all my constraints ",
    backstory = "I constantly travel for work and need to book cheapest flight satisfying all my constraints ",
    tools = [search_tool],
    verbose = True,
    allow_delegation = False,
)

In [ ]:
# Task 1 - Scrap flight prices from cityA to cityB & vice-versa for both direct and indirect flights

flight_scraping_task = Task(
    description = "Get flight prices with all other details like departure/arrival timings, flight code, number of stops "
            "for all flights running within next {trip_window_days} days from todays date "
            " from {cityA} to {cityB} and vice versa i.e. from {cityB} to {cityA}. Get today's date from google ",
    expected_output = " Date, Flight Code, Departure Time, Arrival Time, Duration, No. of stops, flight price "
                        " for both to and fro journey ",
    human_input = True,
    agent = flight_scraper_agent,
)

# Task 2 - Build itinerary for my upcoming trip from cityA to cityB
# Constraints - 1) Morning flights in onward leg and evening/night flights in return leg
# 2) Direct flights only
# 3) My duration of stay in cityB is respected
# 4) Suggest cheapest flights satisfying the above criteria

itinerary_building_task = Task(
    description = " Build a itinerary and suggest which flight on which date should I take from {cityA} to {cityB} "
            " and also when should I return back from {cityB} to {cityA}. "
            " Ensure that I take the cheapest direct flights in my journey from {cityA} to {cityB} during morning "
            " and cheapest direct evening/night flights during return journey from {cityB} to {cityA}." 
            " I need to stay in {cityB} for {trip_duration} days ",
    expected_output = " For both to and fro journeys I should get one or more flights options which includes "
                      " Journey date, Flight Details, Price ",
    human_input = True,
    agent = itinerary_builder_agent,
)

In [ ]:
itinerary_crew = Crew(
    agents = [flight_scraper_agent, itinerary_builder_agent],
    tasks = [flight_scraping_task, itinerary_building_task],
    verbose = True,
)

In [ ]:
parameters = {
    "cityA": "Delhi, India",
    "cityB": "Bangalore, India",
    "trip_duration": 7, # days
    "trip_window_days" : 30, # days
}

In [ ]:
result = itinerary_crew.kickoff(inputs=parameters)